Sequence-to-sequence models are best suited for tasks revolving around generating new sentences depending on a given input, such as summarization, translation, or generative question answering.

# Translation
https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt

KDE4 dataset: https://huggingface.co/datasets/kde4

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

Found cached dataset kde4 (C:/Users/lkk68/.cache/huggingface/datasets/kde4/en-fr-lang1=en,lang2=fr/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [2]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

Loading cached split indices for dataset at C:\Users\lkk68\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-496be247a58b47c1.arrow and C:\Users\lkk68\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-2c0faebb61cdd12e.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [3]:
# rename the "test" key to "validation" 
split_datasets["validation"] = split_datasets.pop("test")

In [5]:
#one element
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [4]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Par défaut pour les threads élargis'}]

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [6]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [7]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [8]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Loading cached processed dataset at C:\Users\lkk68\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-025212206c0a73ce.arrow
Loading cached processed dataset at C:\Users\lkk68\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-dcb99a8d3b4ca5c2.arrow


In [9]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

DataCollatorWithPadding only pads the inputs (input IDs, attention mask, and token type IDs). Our labels should also be padded to the maximum length encountered in the labels. the padding value used to pad the labels should be -100 and not the padding token of the tokenizer, to make sure those padded values are ignored in the loss computation.

This is all done by a DataCollatorForSeq2Seq. Like the DataCollatorWithPadding, it takes the tokenizer used to preprocess the inputs, but it also takes the model. This is because this data collator will also be responsible for preparing the decoder input IDs, which are shifted versions of the labels with a special token at the beginning. Since this shift is done slightly differently for different architectures, the DataCollatorForSeq2Seq needs to know the model object:

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [11]:
#To test this on a few samples
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [14]:
batch["labels"] #our labels have been padded to the maximum length of the batch, using -100:

tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]])

In [15]:
batch["decoder_input_ids"] #shifted versions of the labels

tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])

In [16]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]
[1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]


The feature that Seq2SeqTrainer adds to its superclass Trainer is the ability to use the generate() method during evaluation or prediction. During training, the model will use the decoder_input_ids with an attention mask ensuring it does not use the tokens after the token it’s trying to predict, to speed up training.

 The BLEU score evaluates how close the translations are to their labels. It does not measure the intelligibility or grammatical correctness of the model’s generated outputs, but uses statistical rules to ensure that all the words in the generated outputs also appear in the targets.

One weakness with BLEU is that it expects the text to already be tokenized. SacreBLEU, which addresses this weakness (and others) by standardizing the tokenization step. To use this metric, we first need to install the SacreBLEU library: !pip install sacrebleu

In [17]:
! pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/118.9 kB ? eta -:--:--
     -------------------------------------- 118.9/118.9 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
     - -------------------------------------- 0.2/3.9 MB 3.6 MB/s eta 0:00:02
     --- ------------------------------------ 0.4/3.9 MB 3.9 MB/s eta 0:00:01
     ------ --------------------------------- 0.6/3.9 MB 4.2 MB/s eta 0:00:01
     -------- ------------------------------- 0.9/3.9 MB 4.5 MB/s eta 0:00:01
     ----------- ---------------------------- 1.1/3.9 MB 4.7 MB/s eta 0:00:01
     -------------- ------------------------- 1.4/3.9 MB 4.9 MB/s eta 0:00:01
     ----------------- ---------------------- 1.7/3.9 MB 5.1 MB/s eta 0:00:01
     -------------------- ------------------- 2.0/3.9 MB 5.4 MB/s eta 0:00:01
     ------------------------ --------------- 2.4/3.9 MB 5.6 MB/s eta 0:00:01
 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pccm 0.4.7 requires ccimport>=0.3.1, which is not installed.
pccm 0.4.7 requires lark>=1.0.0, which is not installed.


In [12]:
import evaluate

metric = evaluate.load("sacrebleu")

In [19]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [20]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

To get from the model outputs to texts the metric can use, we will use the tokenizer.batch_decode() method. We just have to clean up all the -100s in the labels (the tokenizer will automatically do the same for the padding token):

In [13]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [14]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [15]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [17]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [44]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/lkk688/marian-finetuned-kde4-en-to-fr into local empty directory.


In [45]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.3645007610321045,
 'eval_bleu': 43.895194293860456,
 'eval_runtime': 1583.4896,
 'eval_samples_per_second': 13.273,
 'eval_steps_per_second': 0.208}

In [46]:
trainer.train()

c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.355300
1000,1.206600
1500,1.163000
2000,1.122500
2500,1.115100
3000,1.062800
3500,1.064700
4000,1.025700
4500,1.021400
5000,1.026400


Adding files tracked by Git LFS: ['source.spm', 'target.spm']. This may take a bit of time if the files are large.


TrainOutput(global_step=17736, training_loss=0.9341417604273485, metrics={'train_runtime': 2111.1917, 'train_samples_per_second': 268.789, 'train_steps_per_second': 8.401, 'total_flos': 1.1322351026307072e+16, 'train_loss': 0.9341417604273485, 'epoch': 3.0})

In [47]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 0.856429398059845,
 'eval_bleu': 52.873596622661076,
 'eval_runtime': 1430.3221,
 'eval_samples_per_second': 14.695,
 'eval_steps_per_second': 0.23,
 'epoch': 3.0}

# A custom training loop

To simplify its evaluation part, we define this postprocess() function that takes predictions and labels and converts them to the lists of strings our metric object will expect:

In [18]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [26]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print("Using device:", device)

Using device: cuda


In [31]:
optimizer

AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 2e-05
    lr: 1.9933178261788964e-05
    weight_decay: 0.0
)

In [19]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [57]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [23]:
num_train_epochs

3

In [24]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        #batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    

  0%|          | 0/70935 [00:00<?, ?it/s]

  0%|          | 0/2628 [00:00<?, ?it/s]

epoch 0, BLEU score: 53.40


  0%|          | 0/2628 [00:00<?, ?it/s]

epoch 1, BLEU score: 53.82


  0%|          | 0/2628 [00:00<?, ?it/s]

epoch 2, BLEU score: 53.82


In [25]:
# Save and upload
output_dir='./output'
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
# if accelerator.is_main_process:
#     tokenizer.save_pretrained(output_dir)
#     repo.push_to_hub(
#         commit_message=f"Training in progress epoch {epoch}", blocking=False
#     )

# Translation OPUS Books dataset
https://huggingface.co/docs/transformers/tasks/translation
Finetune T5 on the English-French subset of the OPUS Books dataset to translate English text to French.

In [26]:
! pip install transformers datasets evaluate sacrebleu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


loading the English-French subset of the OPUS Books dataset: https://huggingface.co/datasets/opus_books

In [27]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to C:/Users/lkk68/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [29]:
books = books["train"].train_test_split(test_size=0.2)

In [31]:
books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 25417
    })
})

In [30]:
#one example
books["train"][0]

{'id': '6091',
 'translation': {'en': 'What a stroke was this for poor Jane! who would willingly have gone through the world without believing that so much wickedness existed in the whole race of mankind, as was here collected in one individual.',
  'fr': 'Quel coup pour la pauvre Jane qui aurait parcouru le monde entier sans s’imaginer qu’il existât dans toute l’humanité autant de noirceur qu’elle en découvrait en ce moment dans un seul homme !'}}

In [32]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The preprocessing function you want to create needs to:

Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.
Tokenize the input (English) and target (French) separately because you can’t tokenize French text with a tokenizer pretrained on an English vocabulary.
Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [33]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [34]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [35]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [36]:
import evaluate

metric = evaluate.load("sacrebleu")

In [37]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [38]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [39]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


Cloning https://huggingface.co/lkk688/my_awesome_opus_books_model into local empty directory.


In [40]:
trainer.train()

c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.860700,1.628886,5.467200,17.630700
2,1.805100,1.604983,5.645900,17.624700


TrainOutput(global_step=12710, training_loss=1.8750171556330215, metrics={'train_runtime': 2906.4741, 'train_samples_per_second': 69.96, 'train_steps_per_second': 4.373, 'total_flos': 5000491514068992.0, 'train_loss': 1.8750171556330215, 'epoch': 2.0})

In [41]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [42]:
inputs = tokenizer(text, return_tensors="pt").input_ids

In [43]:
model.device

device(type='cuda', index=0)

In [44]:
outputs = model.generate(inputs.cuda(), max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [45]:
outputs

tensor([[    0,   325,     3, 10912,   154,    51,     9, 16762,  1394, 17126,
           393,    93,     3,  9305,  2229,  2593,  2210, 11488,     7,    20,
             3,    40,    22, 17694,    17,    15,     5,     1]],
       device='cuda:0')

In [46]:
#Decode the generated token ids back into text:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'La levéma partage ses ressources avec des bactéries fixatrices de l’azote.'

# text summarization
https://huggingface.co/learn/nlp-course/chapter7/5?fw=pt
https://huggingface.co/docs/transformers/tasks/summarization

Summarization creates a shorter version of a document or an article that captures all the important information.  Summarization can be:

Extractive: extract the most relevant information from a document.
Abstractive: generate new text that captures the most relevant information.

In [ ]:
BillSum, summarization of US Congressional and California state bills: https://huggingface.co/datasets/billsum

There are several features:

text: bill text.
summary: summary of the bills.
title: title of the bills. features for us bills. ca bills does not have.
text_len: number of chars in text.
sum_len: number of chars in summary.

In [1]:
! pip install transformers datasets evaluate rouge_score

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     --------------------------- ------------ 1.0/1.5 MB 22.0 MB/s eta 0:00:01
     ------------------------------------- -- 1.4/1.5 MB 14.9 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 13.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=632a979bf60759c0d0b2f12811d314e0ead93d9f24cd1e990e814fa78bb99152
  Stored in directory: C:\Users\lkk68\AppData\Local\Temp\pip-ephem-wheel-cache-t73np93p\wheels\9b\3d\39\09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to C:/Users/lkk68/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc. Subsequent calls will reuse this data.


In [4]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [5]:
#Split the dataset into a train and test set with the train_test_split method:
billsum = billsum.train_test_split(test_size=0.2)

In [7]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [6]:
#one example
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 9114 of the Elections Code is amended to read:\n9114.\nExcept as provided in Section 9115, within 30 days from the date of filing of the petition, excluding Saturdays, Sundays, and holidays, the elections official shall examine the petition, and from the records of registration ascertain whether or not the petition is signed by the requisite number of voters. A certificate showing the results of this examination shall be attached to the petition.\nIn determining the number of valid signatures, the elections official may use the duplicate file of affidavits maintained, or may check the signatures against facsimiles of voters’ signatures, provided that the method of preparing and displaying the facsimiles complies with law.\nThe elections official shall notify the proponents of the petition as to the sufficiency or insufficiency of the petition.\nIf the petition is found insufficient, no further

There are two fields that you’ll want to use:

text: the text of the bill which’ll be the input to the model.
summary: a condensed version of text which’ll be the model target.

In [8]:
from transformers import AutoTokenizer

checkpoint = "t5-small" #https://huggingface.co/t5-small
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The preprocessing function you want to create needs to:

Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
Use the keyword text_target argument when tokenizing labels.
Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [9]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [11]:
tokenized_billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 248
    })
})

Using DataCollatorForSeq2Seq. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [13]:
import evaluate

rouge = evaluate.load("rouge") #load the ROUGE metric: https://huggingface.co/spaces/evaluate-metric/rouge

In [14]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [15]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/lkk688/my_awesome_billsum_model into local empty directory.


In [17]:
#start training
trainer.train()

c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.782855,0.128200,0.037300,0.106600,0.106400,19.000000
2,No log,2.572354,0.138000,0.045900,0.113800,0.113700,19.000000
3,No log,2.509431,0.142300,0.051300,0.117200,0.117000,19.000000
4,No log,2.492936,0.142300,0.052000,0.117000,0.116800,19.000000


TrainOutput(global_step=248, training_loss=3.0391001547536542, metrics={'train_runtime': 122.5556, 'train_samples_per_second': 32.279, 'train_steps_per_second': 2.024, 'total_flos': 1070824333246464.0, 'train_loss': 3.0391001547536542, 'epoch': 4.0})

In [18]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [19]:
inputs = tokenizer(text, return_tensors="pt").input_ids

Use the generate() method to create the summarization

In [22]:
model.device

device(type='cuda', index=0)

In [23]:
outputs = model.generate(inputs.cuda(), max_new_tokens=100, do_sample=False)

In [24]:
outputs

tensor([[    0,     8,    86,    89,  6105,   419,  8291,  1983,  1364,     7,
          7744,  2672,  1358,     6,   533,   124,  1358,     6,    11,   827,
          1358,     3,     5,    34,    31,     7,     8,   167,  8299,  1041,
            30,     3, 26074,     8,  3298,  5362,    16,   797,   892,     3,
             5,    34,    31,   195,   987,     8,  6173,    18,  1123,   138,
           189,    63,    11, 11711,    12,   726,    70,  2725,   698,     3,
             5,     1]], device='cuda:0')

In [25]:
#Decode the generated token ids back into text:
tokenizer.decode(outputs[0], skip_special_tokens=True)

"the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. it's the most aggressive action on tackling the climate crisis in American history. it'll ask the ultra-wealthy and corporations to pay their fair share."